In [1]:
import mediapipe as mp
import numpy as np
import cv2
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
import pandas as pd
from tqdm import tqdm
import gc
from operator import add

2023-09-09 18:52:35.294489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 18:52:35.577987: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/kristian/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [5]:
#video_df = spark.read.format("binaryFile").option(
#    "pathGlobFilter", "*.mp4").load('file:///home/kristian/WLASL/videos_organized/*').orderBy("path")

video_df = spark.read.format("binaryFile").option(
    "pathGlobFilter", "*.mp4").load('file:///home/kristian/Capstone/Sign_Videos_DF/*').orderBy("path")

In [6]:
video_df.show(10)

+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/home/kristi...|2023-09-08 14:50:...|868126|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:50:...|876062|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:51:...|903345|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:51:...|779439|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:51:...|819007|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:58:...|770784|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:58:...|807072|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:58:...|709806|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:58:...|730306|[00 00 00 1C 66 7...|
|file:/home/kristi...|2023-09-08 14:59:...|749810|[00 00 00 1C 66 7...|
+--------------------+--------------------+------+--------------

In [7]:
video_df = video_df.withColumn("columnindex",
                               row_number().over(Window().orderBy("path")))

In [8]:
video_df = video_df.drop('length','modification')

In [9]:
video_df2 = video_df.select("path").collect()

In [10]:
#loop to get labels and filename per row
i = 0
target = []
for row in video_df2:
    subdirname = os.path.basename(os.path.dirname(video_df2[i][0]))
    name = os.path.basename(video_df2[i][0]).split('/')[-1]
    target.append([subdirname, name])
    i = i + 1

In [11]:
target

[['AXE', 'AXE_video_1.mp4'],
 ['AXE', 'AXE_video_2.mp4'],
 ['AXE', 'AXE_video_3.mp4'],
 ['AXE', 'AXE_video_4.mp4'],
 ['AXE', 'AXE_video_5.mp4'],
 ['BACKPACK', 'BACKPACK_video_1.mp4'],
 ['BACKPACK', 'BACKPACK_video_2.mp4'],
 ['BACKPACK', 'BACKPACK_video_3.mp4'],
 ['BACKPACK', 'BACKPACK_video_4.mp4'],
 ['BACKPACK', 'BACKPACK_video_5.mp4'],
 ['BASKETBALL', 'BASKETBALL_video_1.mp4'],
 ['BASKETBALL', 'BASKETBALL_video_2.mp4'],
 ['BASKETBALL', 'BASKETBALL_video_3.mp4'],
 ['BASKETBALL', 'BASKETBALL_video_4.mp4'],
 ['BASKETBALL', 'BASKETBALL_video_5.mp4'],
 ['BEE', 'BEE_video_1.mp4'],
 ['BEE', 'BEE_video_2.mp4'],
 ['BEE', 'BEE_video_3.mp4'],
 ['BEE', 'BEE_video_4.mp4'],
 ['BEE', 'BEE_video_5.mp4'],
 ['BELT', 'BELT_video_1.mp4'],
 ['BELT', 'BELT_video_2.mp4'],
 ['BELT', 'BELT_video_3.mp4'],
 ['BELT', 'BELT_video_4.mp4'],
 ['BELT', 'BELT_video_5.mp4'],
 ['BITE', 'BITE_video_1.mp4'],
 ['BITE', 'BITE_video_2.mp4'],
 ['BITE', 'BITE_video_3.mp4'],
 ['BITE', 'BITE_video_4.mp4'],
 ['BITE', 'BITE_video

In [12]:
targetRDD = sc.parallelize([row[0] for row in target])

In [13]:
labelcount = targetRDD.map(lambda w: (w, 1))

In [14]:
labelcount = labelcount.reduceByKey(add)

In [15]:
labelcount.sortBy(lambda x: x[1],ascending=False).collect()

[('NOON', 5),
 ('RAZOR', 5),
 ('THEY', 5),
 ('AXE', 5),
 ('CONFUSED', 5),
 ('LOCK', 5),
 ('MAPLE', 5),
 ('SINK', 5),
 ('BEE', 5),
 ('CHRISTMAS', 5),
 ('DOG', 5),
 ('HOSPITAL', 5),
 ('ROCKINGCHAIR', 5),
 ('BREAKFAST', 5),
 ('DEAF', 5),
 ('DECIDE', 5),
 ('FLOAT', 5),
 ('JEWELRY', 5),
 ('MECHANIC', 5),
 ('DARK', 5),
 ('DOWNSIZE', 5),
 ('EAT', 5),
 ('FINE', 5),
 ('KNIGHT', 5),
 ('PARTY', 5),
 ('RIVER', 5),
 ('SQUEEZE', 5),
 ('DEVELOP', 5),
 ('ELEVATOR', 5),
 ('FOREIGNER', 5),
 ('TWINS', 5),
 ('BELT', 5),
 ('CANCER', 5),
 ('DEMAND', 5),
 ('DINNER', 5),
 ('DRAG', 5),
 ('GUESS', 5),
 ('LUNCH', 5),
 ('MEAT', 5),
 ('MICROSCOPE', 5),
 ('MOVIE', 5),
 ('PATIENT', 5),
 ('SHAVE', 5),
 ('WHATFOR', 5),
 ('BACKPACK', 5),
 ('BASKETBALL', 5),
 ('BITE', 5),
 ('EDIT', 5),
 ('HALLOWEEN', 5),
 ('HURDLE_TRIP', 5)]

In [16]:
video_df2

[Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/AXE/AXE_video_1.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/AXE/AXE_video_2.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/AXE/AXE_video_3.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/AXE/AXE_video_4.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/AXE/AXE_video_5.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/BACKPACK/BACKPACK_video_1.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/BACKPACK/BACKPACK_video_2.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/BACKPACK/BACKPACK_video_3.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/BACKPACK/BACKPACK_video_4.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/BACKPACK/BACKPACK_video_5.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/BASKETBALL/BASKETBALL_video_1.mp4'),
 Row(path='file:/home/kristian/Capstone/Sign_Videos_DF/BASKETBALL/BASKETBALL_v

In [17]:
video_df3 = pd.DataFrame(video_df2, columns=['path'])

In [18]:
video_df3.head()

,path
0,file:/home/kristian/Capstone/Sign_Videos_DF/AX...
1,file:/home/kristian/Capstone/Sign_Videos_DF/AX...
2,file:/home/kristian/Capstone/Sign_Videos_DF/AX...
3,file:/home/kristian/Capstone/Sign_Videos_DF/AX...
4,file:/home/kristian/Capstone/Sign_Videos_DF/AX...


In [19]:
video_df = video_df3.join(pd.DataFrame(target, columns=['target','file']))

In [20]:
video_df.sample(20)

,path,target,file
117,file:/home/kristian/Capstone/Sign_Videos_DF/FL...,FLOAT,FLOAT_video_3.mp4
135,file:/home/kristian/Capstone/Sign_Videos_DF/HO...,HOSPITAL,HOSPITAL_video_1.mp4
227,file:/home/kristian/Capstone/Sign_Videos_DF/SI...,SINK,SINK_video_3.mp4
18,file:/home/kristian/Capstone/Sign_Videos_DF/BE...,BEE,BEE_video_4.mp4
5,file:/home/kristian/Capstone/Sign_Videos_DF/BA...,BACKPACK,BACKPACK_video_1.mp4
41,file:/home/kristian/Capstone/Sign_Videos_DF/CH...,CHRISTMAS,CHRISTMAS_video_2.mp4
238,file:/home/kristian/Capstone/Sign_Videos_DF/TH...,THEY,THEY_video_4.mp4
110,file:/home/kristian/Capstone/Sign_Videos_DF/FI...,FINE,FINE_video_1.mp4
142,file:/home/kristian/Capstone/Sign_Videos_DF/HU...,HURDLE_TRIP,HURDLE_TRIP_video_3.mp4
17,file:/home/kristian/Capstone/Sign_Videos_DF/BE...,BEE,BEE_video_3.mp4


In [21]:
#loop to get labels and filename per row
i = 0
target2 = []
for row in video_df.path:
    subdirname = os.path.basename(os.path.dirname(video_df.path[i]))
    name = os.path.basename(video_df.path[i]).split('/')[-1]
    target2.append([subdirname])
    i = i + 1

In [22]:
target2

[['AXE'],
 ['AXE'],
 ['AXE'],
 ['AXE'],
 ['AXE'],
 ['BACKPACK'],
 ['BACKPACK'],
 ['BACKPACK'],
 ['BACKPACK'],
 ['BACKPACK'],
 ['BASKETBALL'],
 ['BASKETBALL'],
 ['BASKETBALL'],
 ['BASKETBALL'],
 ['BASKETBALL'],
 ['BEE'],
 ['BEE'],
 ['BEE'],
 ['BEE'],
 ['BEE'],
 ['BELT'],
 ['BELT'],
 ['BELT'],
 ['BELT'],
 ['BELT'],
 ['BITE'],
 ['BITE'],
 ['BITE'],
 ['BITE'],
 ['BITE'],
 ['BREAKFAST'],
 ['BREAKFAST'],
 ['BREAKFAST'],
 ['BREAKFAST'],
 ['BREAKFAST'],
 ['CANCER'],
 ['CANCER'],
 ['CANCER'],
 ['CANCER'],
 ['CANCER'],
 ['CHRISTMAS'],
 ['CHRISTMAS'],
 ['CHRISTMAS'],
 ['CHRISTMAS'],
 ['CHRISTMAS'],
 ['CONFUSED'],
 ['CONFUSED'],
 ['CONFUSED'],
 ['CONFUSED'],
 ['CONFUSED'],
 ['DARK'],
 ['DARK'],
 ['DARK'],
 ['DARK'],
 ['DARK'],
 ['DEAF'],
 ['DEAF'],
 ['DEAF'],
 ['DEAF'],
 ['DEAF'],
 ['DECIDE'],
 ['DECIDE'],
 ['DECIDE'],
 ['DECIDE'],
 ['DECIDE'],
 ['DEMAND'],
 ['DEMAND'],
 ['DEMAND'],
 ['DEMAND'],
 ['DEMAND'],
 ['DEVELOP'],
 ['DEVELOP'],
 ['DEVELOP'],
 ['DEVELOP'],
 ['DEVELOP'],
 ['DINNER'],
 ['DINN

In [23]:
video_df = video_df.join(pd.DataFrame(target2, columns=['originaltarget']))

In [24]:
video_df.sample(50)

,path,target,file,originaltarget
55,file:/home/kristian/Capstone/Sign_Videos_DF/DE...,DEAF,DEAF_video_1.mp4,DEAF
115,file:/home/kristian/Capstone/Sign_Videos_DF/FL...,FLOAT,FLOAT_video_1.mp4,FLOAT
212,file:/home/kristian/Capstone/Sign_Videos_DF/RI...,RIVER,RIVER_video_3.mp4,RIVER
173,file:/home/kristian/Capstone/Sign_Videos_DF/ME...,MEAT,MEAT_video_4.mp4,MEAT
11,file:/home/kristian/Capstone/Sign_Videos_DF/BA...,BASKETBALL,BASKETBALL_video_2.mp4,BASKETBALL
137,file:/home/kristian/Capstone/Sign_Videos_DF/HO...,HOSPITAL,HOSPITAL_video_3.mp4,HOSPITAL
139,file:/home/kristian/Capstone/Sign_Videos_DF/HO...,HOSPITAL,HOSPITAL_video_5.mp4,HOSPITAL
93,file:/home/kristian/Capstone/Sign_Videos_DF/DR...,DRAG,DRAG_video_4.mp4,DRAG
134,file:/home/kristian/Capstone/Sign_Videos_DF/HA...,HALLOWEEN,HALLOWEEN_video_5.mp4,HALLOWEEN
154,file:/home/kristian/Capstone/Sign_Videos_DF/KN...,KNIGHT,KNIGHT_video_5.mp4,KNIGHT


In [25]:
video_df['Frequency'] = video_df['target'].map(video_df['target'].value_counts())

In [26]:
video_df.sample(20)

,path,target,file,originaltarget,Frequency
66,file:/home/kristian/Capstone/Sign_Videos_DF/DE...,DEMAND,DEMAND_video_2.mp4,DEMAND,5
173,file:/home/kristian/Capstone/Sign_Videos_DF/ME...,MEAT,MEAT_video_4.mp4,MEAT,5
172,file:/home/kristian/Capstone/Sign_Videos_DF/ME...,MEAT,MEAT_video_3.mp4,MEAT,5
71,file:/home/kristian/Capstone/Sign_Videos_DF/DE...,DEVELOP,DEVELOP_video_2.mp4,DEVELOP,5
102,file:/home/kristian/Capstone/Sign_Videos_DF/ED...,EDIT,EDIT_video_3.mp4,EDIT,5
97,file:/home/kristian/Capstone/Sign_Videos_DF/EA...,EAT,EAT_video_3.mp4,EAT,5
5,file:/home/kristian/Capstone/Sign_Videos_DF/BA...,BACKPACK,BACKPACK_video_1.mp4,BACKPACK,5
8,file:/home/kristian/Capstone/Sign_Videos_DF/BA...,BACKPACK,BACKPACK_video_4.mp4,BACKPACK,5
108,file:/home/kristian/Capstone/Sign_Videos_DF/EL...,ELEVATOR,ELEVATOR_video_4.mp4,ELEVATOR,5
4,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_5.mp4,AXE,5


In [27]:
video_df['target'].unique()

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE_TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [28]:
video_df['target'].nunique()

50

In [29]:
video_df['target'] = video_df['target'].replace('HURDLE_TRIP', 'HURDLE/TRIP')

In [30]:
video_df['target'].unique()

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [31]:
video_df.sample(100)

,path,target,file,originaltarget,Frequency
88,file:/home/kristian/Capstone/Sign_Videos_DF/DO...,DOWNSIZE,DOWNSIZE_video_4.mp4,DOWNSIZE,5
37,file:/home/kristian/Capstone/Sign_Videos_DF/CA...,CANCER,CANCER_video_3.mp4,CANCER,5
142,file:/home/kristian/Capstone/Sign_Videos_DF/HU...,HURDLE/TRIP,HURDLE_TRIP_video_3.mp4,HURDLE_TRIP,5
132,file:/home/kristian/Capstone/Sign_Videos_DF/HA...,HALLOWEEN,HALLOWEEN_video_3.mp4,HALLOWEEN,5
126,file:/home/kristian/Capstone/Sign_Videos_DF/GU...,GUESS,GUESS_video_2.mp4,GUESS,5
...,...,...,...,...,...
175,file:/home/kristian/Capstone/Sign_Videos_DF/ME...,MECHANIC,MECHANIC_video_1.mp4,MECHANIC,5
232,file:/home/kristian/Capstone/Sign_Videos_DF/SQ...,SQUEEZE,SQUEEZE_video_3.mp4,SQUEEZE,5
248,file:/home/kristian/Capstone/Sign_Videos_DF/WH...,WHATFOR,WHATFOR_video_4.mp4,WHATFOR,5
201,file:/home/kristian/Capstone/Sign_Videos_DF/PA...,PATIENT,PATIENT_video_2.mp4,PATIENT,5


In [32]:
video_df = video_df.drop(['originaltarget'], axis=1)

In [33]:
video_df.rename(columns={'target': 'Gloss'}, inplace=True)

In [34]:
video_df.sample(10)

,path,Gloss,file,Frequency
190,file:/home/kristian/Capstone/Sign_Videos_DF/NO...,NOON,NOON_video_1.mp4,5
31,file:/home/kristian/Capstone/Sign_Videos_DF/BR...,BREAKFAST,BREAKFAST_video_2.mp4,5
215,file:/home/kristian/Capstone/Sign_Videos_DF/RO...,ROCKINGCHAIR,ROCKINGCHAIR_video_1.mp4,5
59,file:/home/kristian/Capstone/Sign_Videos_DF/DE...,DEAF,DEAF_video_5.mp4,5
76,file:/home/kristian/Capstone/Sign_Videos_DF/DI...,DINNER,DINNER_video_2.mp4,5
84,file:/home/kristian/Capstone/Sign_Videos_DF/DO...,DOG,DOG_video_5.mp4,5
121,file:/home/kristian/Capstone/Sign_Videos_DF/FO...,FOREIGNER,FOREIGNER_video_2.mp4,5
224,file:/home/kristian/Capstone/Sign_Videos_DF/SH...,SHAVE,SHAVE_video_5.mp4,5
54,file:/home/kristian/Capstone/Sign_Videos_DF/DA...,DARK,DARK_video_5.mp4,5
120,file:/home/kristian/Capstone/Sign_Videos_DF/FO...,FOREIGNER,FOREIGNER_video_1.mp4,5


In [35]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return frames, fps, length, width, height

In [36]:
words = ['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR']

In [37]:
video_df.head()

,path,Gloss,file,Frequency
0,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_1.mp4,5
1,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_2.mp4,5
2,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_3.mp4,5
3,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_4.mp4,5
4,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_5.mp4,5


In [38]:
video_df.tail()

,path,Gloss,file,Frequency
245,file:/home/kristian/Capstone/Sign_Videos_DF/WH...,WHATFOR,WHATFOR_video_1.mp4,5
246,file:/home/kristian/Capstone/Sign_Videos_DF/WH...,WHATFOR,WHATFOR_video_2.mp4,5
247,file:/home/kristian/Capstone/Sign_Videos_DF/WH...,WHATFOR,WHATFOR_video_3.mp4,5
248,file:/home/kristian/Capstone/Sign_Videos_DF/WH...,WHATFOR,WHATFOR_video_4.mp4,5
249,file:/home/kristian/Capstone/Sign_Videos_DF/WH...,WHATFOR,WHATFOR_video_5.mp4,5


In [39]:
video_df.describe()

,Frequency
count,250.0
mean,5.0
std,0.0
min,5.0
25%,5.0
50%,5.0
75%,5.0
max,5.0


In [40]:
video_df = video_df.sort_values(by='Gloss')
video_df = video_df.reset_index(drop=True)

In [41]:
video_df.head()

,path,Gloss,file,Frequency
0,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_1.mp4,5
1,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_2.mp4,5
2,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_3.mp4,5
3,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_4.mp4,5
4,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_5.mp4,5


In [42]:
tqdm.pandas()
video_df[['Frames', 'FPS', 'Length', 'Width', 'Height']] = video_df['path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████████| 250/250 [00:06<00:00, 36.07it/s]


In [43]:
video_df.head()

,path,Gloss,file,Frequency,Frames,FPS,Length,Width,Height
0,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_1.mp4,5,130,24,5,640,480
1,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_2.mp4,5,130,24,5,640,480
2,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_3.mp4,5,130,24,5,640,480
3,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_4.mp4,5,130,24,5,640,480
4,file:/home/kristian/Capstone/Sign_Videos_DF/AX...,AXE,AXE_video_5.mp4,5,130,24,5,640,480


In [44]:
video_df.describe()

,Frequency,Frames,FPS,Length,Width,Height
count,250.0,250.0,250.0,250.0,250.0,250.0
mean,5.0,130.0,24.0,5.0,640.0,480.0
std,0.0,0.0,0.0,0.0,0.0,0.0
min,5.0,130.0,24.0,5.0,640.0,480.0
25%,5.0,130.0,24.0,5.0,640.0,480.0
50%,5.0,130.0,24.0,5.0,640.0,480.0
75%,5.0,130.0,24.0,5.0,640.0,480.0
max,5.0,130.0,24.0,5.0,640.0,480.0


In [45]:
cap = cv2.VideoCapture(video_df.path[3])
#cap = cv2.VideoCapture(0)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Display the annotated frame
        cv2.imshow("WLASL", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [46]:
video_df.path[1]

'file:/home/kristian/Capstone/Sign_Videos_DF/AXE/AXE_video_2.mp4'

In [47]:
video_df.to_csv('Signs_DF.csv', index=False)